In [7]:
from typing import Optional, List, Tuple, Dict
from dotenv import load_dotenv, find_dotenv
from datetime import datetime
from openai import OpenAI
import requests
import json

# Functions

### WMO Weather interpretation codes (WW)

In [ ]:
def get_weather_description(code: int , city_location: Optional[str] = "") -> Optional[str]:
    # Central_Europe_cities
    central_europe_cities = ["Berlin","Vienna","Prague","Budapest","Warsaw","Bratislava","Ljubljana","Zagreb","Munich","Frankfurt","Zurich","Geneva","Milan","Rome","Madrid","Paris","Brussels","Amsterdam" # Add more cities as needed
    ]
    # Weather Codes
    weather_codes = {
        0: "Clear sky",
        1: "Mainly clear",
        2: "Partly cloudy",
        3: "Overcast",
        45: "Fog",
        48: "Depositing rime fog",
        51: "Drizzle with Light intensity",
        53: "Drizzle with Moderate intensity",
        55: "Drizzle with Dense intensity",
        56: "Freezing Drizzle: Light intensity",
        57: "Freezing Drizzle: Dense intensity",
        61: "Rain with Slight intensity",
        63: "Rain with Moderate intensity",
        65: "Rain with Heavy intensity",
        66: "Freezing Rain with Light intensity",
        67: "Freezing Rain with Heavy intensity",
        71: "Snowfall with Slight intensity",
        73: "Snowfall with Moderate intensity",
        75: "Snowfall with Heavy intensity",
        77: "Snow grains",
        80: "Rain showers with Slight intensity",
        81: "Rain showers with Moderate intensity",
        82: "Rain showers with Violent intensity",
        85: "Snow showers with Slight",
        86: "Snow showers with Heavy",
        95: "Thunderstorm with Slight or moderate",
        96: "Thunderstorm with slight hail",
        99: "Thunderstorm with heavy hail"
    }

    # Check if the code is in the dictionary
    if code in weather_codes:
        if code in [95, 96, 99] and city_location not in central_europe_cities:
            return "Thunderstorm with hail warning is not available outside Central Europe"
        else:
            return weather_codes[code]
    else:
        return "Unknown weather code"

In [6]:
# Example usage:
weather_code_input = 67
city_location_input = "Berlin"  # Change to the actual city name
weather_description = get_weather_description(weather_code_input, city_location_input)
print(f"Weather description: {weather_description}")


Weather description: Freezing Rain: Heavy intensity


In [ ]:
# Example usage:
weather_code_input = 95
city_location_input = "Berlin"  # Change to the actual city name
weather_description = get_weather_description(weather_code_input, city_location_input)
print(f"Weather description: {weather_description}")

### Get Latitude and Longitude of the City

In [ ]:
# Get Latitude and Longitude of the City
def get_lat_long_from_city(city_name: str, count: int = 1, language: str = 'en', format: str = 'json') -> Optional[Tuple[float, float]]:
    api_url: str = "https://geocoding-api.open-meteo.com/v1/search"
    params = {
        'name': city_name,
        'count': count,
        'language': language,
        'format': format
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Raise an exception for bad responses (4xx and 5xx)
        result = response.json()
        if result:
            longitude = result['results'][0]['longitude']
            latitude = result['results'][0]['latitude']
            return float(longitude), float(latitude)
        else:
            print(f"Error: Unable to retrieve coordinates. API response: {result}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request Exception: {e}")
        return None
    except Exception as e:
        print(f"Exception: {e}")
        return None

In [9]:
# Example usage
city_name:str = "Lahore"
coordinates = get_lat_long_from_city(city_name)
if coordinates:
    Longitude, Latitude = coordinates
    print("Longitude", Longitude)
    print("Latitude", Latitude)

# print("Longitude", Longitude)
# print("Latitude",Latitude)

Longitude 74.35071
Latitude 31.558


### UNIX Timestamp to date and time

In [15]:
from datetime import datetime, timezone

timestamp1 = 1702339200
timestamp2 = 1702508400

dt_object = datetime.fromtimestamp(timestamp2, timezone.utc)
formatted_date_time = dt_object.strftime(f"%d-%m-%Y %H:%M")
print(formatted_date_time)


13-12-2023 23:00
